In [8]:
import pandas as pd
from collections import defaultdict


In [9]:
df = pd.read_csv('./monitor/logs/packets_log.csv')
df = df.sort_values('Time')
df

,Node,Packet,Direction,Protocol,Port,Time,Size,Offset
31845,0,0,TX,UDP,520,0.049858,52,0
31846,0,0,TX,UDP,520,0.049858,52,0
31847,0,0,TX,UDP,520,0.049858,52,0
31848,0,0,TX,UDP,520,0.049858,52,0
31849,0,0,TX,UDP,520,0.049858,52,0
...,...,...,...,...,...,...,...,...
43846,0,1800,RX,UDP,520,99.660900,212,0
43847,4,1803,RX,UDP,520,99.660900,212,0
43848,12,1802,RX,UDP,520,99.665900,212,0
43849,2,1806,RX,UDP,520,99.667200,212,0


In [10]:
tx_packets = {}  # {packet_id: (sender_node, tx_time)}
link_stats = defaultdict(lambda: {
    'sent': 0,
    'received': 0,
    'lost': 0,
    'sent_bytes' : 0,
    'received_byte' :0,
    'times': []
})

matched_rx_indices = set()

In [ ]:
for index, row in df.iterrows():
    if row['Direction'] == 'TX' and row['Port'] not in [520,521]:
        tx_node = row['Node']
        tx_packet = row['Packet']
        tx_time = row['Time']
        
        tx_packets[tx_packet] = (tx_node, tx_time)
        
        rx_candidates = df[
            (df['Packet'] == tx_packet) &
            (df['Direction'] == 'RX') &
            (df['Node'] != tx_node) &
            (~df.index.isin(matched_rx_indices))
        ]
        
        if not rx_candidates.empty:
            rx_index = rx_candidates.index[0]
            rx_node = rx_candidates.iloc[0]['Node']
            rx_time = rx_candidates.iloc[0]['Time']
            rx_bytes = rx_candidates.iloc[0]['Size']
            
            link = (tx_node, rx_node)
            link_stats[link]['sent'] += 1
            link_stats[link]['received'] += 1
            link_stats[link]['sent_bytes'] += row['Size']
            link_stats[link]['received_byte'] += rx_bytes
            link_stats[link]['times'].append(rx_time - tx_time)
            
            matched_rx_indices.add(rx_index)
        else:
            link_stats[(tx_node, 'UNKNOWN')]['lost'] += 1

    elif row['Direction'] == 'RX' and index not in matched_rx_indices:
        # Handle unmatched RX packets (optional)
        pass
    


In [ ]:
for (sender, receiver), stats in link_stats.items():
    if receiver != 'UNKNOWN':
        # Lost = Sent by sender intended for receiver but not received
        # This is simplified; adjust based on your logic
        stats['lost'] = stats['sent'] - stats['received']
    else:
        stats['lost'] = stats['sent']

In [ ]:
threshold = 0
for (sender, receiver), stats in link_stats.items():
    if receiver == 'UNKNOWN':
        print(f"Node {sender} Lost Packets (Unknown Receiver): {stats['lost']}")
    else:
        print(f"Link {sender} -> {receiver}:")
        print(f"  Sent: {stats['sent']}")
        print(f"  Received: {stats['received']}")
        print(f"  Lost: {stats['lost']}")
        if stats['received'] > 0:
            avg_time = sum(stats['times'])
            if stats['lost'] > 0:
                weighted_time = sum(stats['times']) / stats['received'] + threshold / stats['lost']
            tp = stats['received'] / sum(stats['times'])
            print(f"  Average Time: {avg_time:.6f}s")
            print(f"  Throughput: {tp:.6f}p/s")
        else:
            print("  No successful transmissions")
        print("-------------------")



In [ ]:
for (sender, receiver), stats in link_stats.items():
    if receiver == 'UNKNOWN':
        print(f"Node {sender} Lost Packets: {stats['lost']}")
    else:
        print(f"Link {sender} -> {receiver}:")
        print(f"  Sent: {stats['sent']}")
        print(f"  Received: {stats['received']}")
        print(f"  Lost: {stats['lost']}")
        print(f"  Sent_Bytes: {stats['sent_bytes']}")
        print(f"  Received_Bytes: {stats['received_byte']}")
        if stats['received'] > 0:
            avg_time = sum(stats['times']) / len(stats['times'])
            total_time = sum(stats['times'])
            throughput = stats['received'] / total_time  # Packets per second
            tp2 = stats['received_byte'] / total_time
            print(f"  Total Time: {total_time:.6f}s")
            print(f"  Average Time: {avg_time:.6f}s")
            print(f"  Throughput: {throughput:.6f} packets/s")
            print(f"  Throughput2: {tp2*8/10**6:.6f} Mbps")

        print("-------------------")